# Create the Haifa Map

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
import mayavi.mlab as mlab
import os
import pandas as pd
from scipy.interpolate import griddata, interp2d, RectBivariateSpline
import tqdm

### The trick below is needed as tvtk steals the stdout of the notebook

In [12]:
import sys
old_stdout = sys.stdout
from tvtk.api import tvtk
sys.stdout = old_stdout

## Create the Haifa map from Tiles

In [3]:
df = pd.read_csv("haifa_map_tiles/tiles_list.txt", sep="\s+")

In [4]:
def find_limits(Axis, axis):
    """Return indices of X/Ymap to use when interpolating from x/ymap.
    Check if Xmap[0] is in xmap.
    If it does xlimit = 0
    if not find xmap[0] in Xmap.
    Repeat for Xmap[-1] and xmap[-1]"""

    limits = [0, 0]
    if np.searchsorted(axis, Axis[0]) > 0:
        limits[0] = 0
    else:
        limits[0] = np.searchsorted(Axis, axis[0])
        
    if np.searchsorted(axis, Axis[-1]) < len(axis):
        limits[1] = len(Axis) - 1
    else:
        limits[1] = np.searchsorted(Axis, axis[-1]) - 1
        
    return limits

In [5]:
ymap, xmap = np.linspace(32, 33, 1200), np.linspace(34, 36, 2400)
Ymap, Xmap = np.meshgrid(ymap, xmap, indexing="ij")
map_img = np.dstack([np.zeros_like(Xmap) for _ in range(3)])

for _, row in tqdm.tqdm(df.iterrows()):
    path = os.path.join("haifa_map_tiles", row["ImageFileName"][:-1])
    
    t = row["Top_Edge_Latitude"]
    b = row["Bottom_Edge_Latitude"]
    l = row["Left_Edge_Longitude"]
    r = row["Right_Edge_Longitude"]

    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)[::-1, ...]
    h, w, _ = img.shape
    
    x = np.linspace(l, r, w)
    y = np.linspace(b, t, h)
    
    ylimits = find_limits(ymap, y)
    xlimits = find_limits(xmap, x)

    for i in range(3):
        interp = RectBivariateSpline(y, x, img[..., i])
        map_img[ylimits[0]:ylimits[1], xlimits[0]:xlimits[1],i] = \
            interp(ymap[ylimits[0]:ylimits[1]], xmap[xlimits[0]:xlimits[1]])
            
map_img = map_img[::-1, ...].astype(np.uint8)

In [6]:
cv2.imwrite("haifa_map_tiles/haifa_map.jpg", cv2.cvtColor(map_img, cv2.COLOR_RGB2BGR))

True

## Show the map

In [8]:
import matplotlib.mlab as ml
import numpy as np
import pymap3d


def loadMapData():
    """Load height data for map visualization."""

    path1 = r"data\N32E034.hgt"
    path2 = r"data\N32E035.hgt"
    path3 = r"haifa_map_tiles\haifa_map.jpg"

    with open(path1) as hgt_data:
        hgt1 = np.fromfile(hgt_data, np.dtype('>i2')).reshape((1201, 1201))[:1200, :1200]
    with open(path2) as hgt_data:
        hgt2 = np.fromfile(hgt_data, np.dtype('>i2')).reshape((1201, 1201))[:1200, :1200]
    hgt = np.hstack((hgt1, hgt2)).astype(np.float32)
    lon, lat = np.meshgrid(np.linspace(34, 36, 2400, endpoint=False), np.linspace(32, 33, 1200, endpoint=False)[::-1])
    map_texture = cv2.cvtColor(cv2.imread(path3), cv2.COLOR_BGR2RGB)
    
    return lat[100:400, 1100:1400], lon[100:400, 1100:1400], hgt[100:400, 1100:1400], map_texture[100:400, 1100:1400, ...]


def calcSeaMask(hgt_array):
    """Calc a masking to the sea.

    Note:
        This code is uses empirical magic number, and should be adjusted if
        grid sizes change.
    """

    hgt_u8 = (255 * (hgt_array - hgt_array.min()) / (hgt_array.max() - hgt_array.min())).astype(np.uint8)

    mask = (hgt_u8 > 7).astype(np.uint8)*255
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_open)
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (16, 16))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_close)
    mask[250:, 250:] = 255

    return mask < 255


def convertMapData(lat, lon, hgt, map_texture, lat0=32.775776, lon0=35.024963, alt0=229):
    """Convert lat/lon/height data to grid data."""

    n, e, d = pymap3d.geodetic2ned(
        lat, lon, hgt,
        lat0=lat0, lon0=lon0, h0=alt0)

    x, y, z = e, n, -d

    xi = np.linspace(-10000, 10000, 300)
    yi = np.linspace(-10000, 10000, 300)
    X, Y = np.meshgrid(xi, yi)

    print(y.flatten().shape, x.flatten().shape, map_texture[..., 0].flatten().shape)
    Z = ml.griddata(y.flatten(), x.flatten(), z.flatten(), yi, xi, interp='linear')
    R = ml.griddata(y.flatten(), x.flatten(), map_texture[..., 0].flatten(), yi, xi, interp='linear')
    G = ml.griddata(y.flatten(), x.flatten(), map_texture[..., 1].flatten(), yi, xi, interp='linear')
    B = ml.griddata(y.flatten(), x.flatten(), map_texture[..., 2].flatten(), yi, xi, interp='linear')
    
    Z_mask = calcSeaMask(Z)

    return X, Y, Z, Z_mask, np.dstack((R, G, B))

In [9]:
map_coords = loadMapData()
X, Y, Z, Z_mask, map_texture = convertMapData(
    map_coords[0],
    map_coords[1],
    map_coords[2],
    map_coords[3],
)

In [10]:
tmp_filename=r'haifa_map_tiles\haifa_map_tmp.jpg'
map_array = np.rot90(map_texture, k=1).astype(np.uint8)
cv2.imwrite(tmp_filename, cv2.cvtColor(map_array, cv2.COLOR_RGB2BGR))

True

In [13]:
img = tvtk.JPEGReader(file_name=tmp_filename)
texture = tvtk.Texture(input_connection=img.output_port, interpolate=1)

s = mlab.surf(Y, X, Z, color=(1.,1.,1.))
s.actor.actor.mapper.scalar_visibility = False
s.actor.enable_texture = True
s.actor.tcoord_generator_mode = 'plane'
s.actor.actor.texture = texture

mlab.show()